In [2]:
from transformers import DecisionTransformerModel, DecisionTransformerConfig
import numpy as np
import torch

device = "cuda"

scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 3600 / scale  # evaluation conditioning targets, 3600 is reasonable from the paper LINK
state_dim = (3, 32, 32)
state_dim_flatten = state_dim[0]*state_dim[1]*state_dim[2]
act_dim = 6

/home/pedron18/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
n_positions=1024*4
max_ep_len=1024 # no impact?

t = 1# max int(n_positions/3) # number of elements in sequence
b = 1 # batch 

#rewards = torch.rand((b,2, t), device=device, dtype=torch.float32) # doesnt matter idk why exists

target_return = torch.rand((b, t, 1), device=device, dtype=torch.float32) # expected future return
states = torch.rand((b, t, state_dim_flatten), device=device, dtype=torch.float32)#.reshape(1, 1, state_dim_flatten) #
actions = torch.rand((b, t, act_dim), device=device, dtype=torch.float32)
timesteps = torch.randint_like(torch.zeros((b, t)), device=device, low = 100, high = max_ep_len, dtype=torch.long) # integer what timestep we on



#target_return, states, actions, timesteps = [1],  torch.rand((t, state_dim_flatten), device=device), [], [1]
#target_return, states, actions, timesteps = torch.tensor(target_return, device=device), states, torch.tensor(actions, device=device), torch.tensor(timesteps, device=device)

attention_mask = torch.ones((b, t), device=device, dtype=torch.long) # this is the default

print("target_return",target_return.shape)
print("states",states.shape)
print("actions",actions.shape)
#print("rewards",rewards.shape)
print("timesteps", timesteps.shape)
print("attention_mask", attention_mask.shape)

config = DecisionTransformerConfig(state_dim_flatten, act_dim, max_ep_len=max_ep_len,n_positions=n_positions)


model = DecisionTransformerModel(config).to(device)
model.eval()
with torch.no_grad():
        state_preds, action_preds, return_preds = model(states=states,
                actions=actions,
                rewards=None,
                returns_to_go=target_return,
                timesteps=timesteps,
                attention_mask=attention_mask,
                return_dict=False)

        #print("argmax", torch.argmax(action_preds).item())
        print("state_preds",state_preds.shape)
        print("action_preds",action_preds.shape)
        print("return_preds", return_preds.shape)


        print("state_preds", state_preds[0])
        print("action_preds", action_preds[0])
        print("return_preds", return_preds[0])

target_return torch.Size([1, 1, 1])
states torch.Size([1, 1, 3072])
actions torch.Size([1, 1, 6])
timesteps torch.Size([1, 1])
attention_mask torch.Size([1, 1])
state_preds torch.Size([1, 1, 3072])
action_preds torch.Size([1, 1, 6])
return_preds torch.Size([1, 1, 1])
state_preds tensor([[-0.2208, -0.6347, -0.0118,  ...,  0.2574, -0.2670, -0.0992]],
       device='cuda:0')
action_preds tensor([[-0.1034,  0.0587, -0.2295, -0.0095,  0.5759, -0.1647]],
       device='cuda:0')
return_preds tensor([[-0.2693]], device='cuda:0')


In [4]:
import random

population = [0, 1, 2, 3, 4]
ls = random.sample(population,5)

print("Pure random: ", ls)

max_sequence = 4

def sample_with_order(population, max_sequence_length):
    """
        Samples [1,population] elements from a sequence mantaining the order
    """
    k = random.randint(1, max_sequence_length)
    k_start = random.randint(0, len(population)-k)
    return population[k_start: k_start+k]


print(sample_with_order(population, max_sequence))

   

Pure random:  [3, 1, 4, 2, 0]
[0, 1]


In [5]:
target_return = torch.ones(1, device=device, dtype=torch.float32).reshape(100, 1)

RuntimeError: shape '[100, 1]' is invalid for input of size 1